In [1]:
import os
import cv2
import pandas as pd
import numpy as np

In [2]:
len(os.listdir('/OFA/imgs_test_public'))

1705

In [ ]:
%cd yolov7/

In [3]:
def get_new_coords(x_center, y_center, w, h):
    x_left = int(x_center - w / 2.)
    x_right = int(x_center + w / 2.)
    y_top = int(y_center - h / 2.)
    y_bot = int(y_center + h / 2.)
    return (x_left, y_top), (x_right, y_bot)

In [4]:
def get_iou(bb1, bb2):
    # Taken from https://stackoverflow.com/a/42874377
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    bb1 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x, y) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
    """
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [5]:
test_public = pd.read_csv('ofa_test/test_public.csv')
test_public.head()

,image,width,height,left,top,right,bottom,question
0,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,500,375,130,160,217,367,Where can we cook meals
1,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,480,640,352,479,447,512,what do we use to make and receive calls?
2,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,500,375,318,68,499,241,Where can we watch news?
3,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,640,426,130,135,377,275,What is used to change TV channels?
4,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,640,472,224,0,268,47,What is used to carry more belongings?


In [6]:
labels_path = 'yolov7/runs/detect/exp/labels/'
color = (255, 0, 0)

test_public_iou = [0 for i in range(len(test_public))]

for id, row in test_public.iterrows():
    # print(row[1]['image'])
    img_path = row['image'][-16:]
    # print(img_path)
    im = cv2.imread('/OFA/imgs_test_public/' + img_path)
    lbl = labels_path + img_path[:-3] +'txt'
    right_true = row['right']
    bottom_true = row['bottom']
    left_true = row['left']
    top_true = row['top']
    bb_true = {'x1': left_true, 'y1': top_true, 'x2': right_true, 'y2': bottom_true}
    true_img = cv2.rectangle(im, (bb_true['x1'], bb_true['y1']), (bb_true['x2'], bb_true['y2']), color)
    # print(row[1]['question'])
    # cv2.imshow('true_img', true_img)
    # if lbl == '/content/yolor/runs/detect/exp6/labels/000000001066.txt':
        # print(lbl)
    try:
        with open(lbl) as f:
            # print(im.shape)
            img_height, img_width, _  = im.shape
            lines = f.readlines()
            best_iou = 0.
            for line in lines:
                _, x_center, y_center, w, h = line.split(' ')
                _, x_center, y_center, w, h = int(_), float(x_center) * img_width, float(y_center) * img_height, float(w) * img_width, float(h) * img_height
                start_point, end_point = get_new_coords(x_center, y_center, w, h)
                new_img = cv2.rectangle(im, start_point, end_point, color)
                # print(start_point, end_point, h, w)
                # cv2.imshow('predict_img', new_img)
                pred_bb = {'x1': start_point[0], 'y1': start_point[1], 'x2': end_point[0], 'y2': end_point[1]}
                cur_iou = get_iou(bb_true, pred_bb)
                if cur_iou > best_iou:
                    best_iou = cur_iou
            test_public_iou[id] = best_iou
    except:
        print(f'Did not found {lbl} file')

        # print(get_iou(bb_true, pred_bb))
      # break

Did not found /media/hdd/anoskov/ofa_test/yolov7/runs/detect/exp/labels/000000070028.txt file
Did not found /media/hdd/anoskov/ofa_test/yolov7/runs/detect/exp/labels/000000092387.txt file
Did not found /media/hdd/anoskov/ofa_test/yolov7/runs/detect/exp/labels/000000166149.txt file
Did not found /media/hdd/anoskov/ofa_test/yolov7/runs/detect/exp/labels/000000254415.txt file
Did not found /media/hdd/anoskov/ofa_test/yolov7/runs/detect/exp/labels/000000388635.txt file
Did not found /media/hdd/anoskov/ofa_test/yolov7/runs/detect/exp/labels/000000404627.txt file
Did not found /media/hdd/anoskov/ofa_test/yolov7/runs/detect/exp/labels/000000417535.txt file
Did not found /media/hdd/anoskov/ofa_test/yolov7/runs/detect/exp/labels/000000419407.txt file
Did not found /media/hdd/anoskov/ofa_test/yolov7/runs/detect/exp/labels/000000468002.txt file
Did not found /media/hdd/anoskov/ofa_test/yolov7/runs/detect/exp/labels/000000478104.txt file
Did not found /media/hdd/anoskov/ofa_test/yolov7/runs/detect

In [7]:
for i in range(10, 101, 10):
    print(f'Pecrentle {i}-th : {np.percentile(test_public_iou, i)}')
print(f'Mean value : {np.mean(test_public_iou)}')

Pecrentle 10-th : 0.0
Pecrentle 20-th : 0.0095543644895592
Pecrentle 30-th : 0.03505737721877075
Pecrentle 40-th : 0.11199060959598878
Pecrentle 50-th : 0.6715288966509648
Pecrentle 60-th : 0.8551993214588633
Pecrentle 70-th : 0.9125793166564978
Pecrentle 80-th : 0.9411641753424994
Pecrentle 90-th : 0.9654366182204315
Pecrentle 100-th : 1.0
Mean value : 0.497429095813356


Pecrentle 40-th : 0.11199060959598878
Pecrentle 50-th : 0.6715288966509648
 --conf 0.25



## Baseline with CLIP

In [11]:
import clip
import shutil
import os
from tqdm.auto import tqdm
import torch
from PIL import Image
import matplotlib.pyplot as plt

In [9]:
device = 'cuda:1'

In [14]:
def process_image(img_path, bb_prediction, preprocess):
    np_img = cv2.imread(img_path)
    imgs = []
    with open(bb_prediction) as f:
        lines = f.readlines()
    bbs = [list(map(float, line.split(' ')[1:5])) for line in lines]
    bbs_processed = []
    for bb in bbs:
        y, x, h, w = bb
        height, width, channels = np_img.shape
        x *= height
        w *= height
        y *= width
        h *= width
        crop = np_img[int(x - w // 2):int(x + w // 2), int(y - h // 2):int(y + h // 2), :]
        imgs.append(preprocess(Image.fromarray(crop)).unsqueeze(0).to(device))
        bbs_processed.append({'y1': int(x - w // 2), 'y2': int(x + w // 2), 'x1': int(y - h // 2), 'x2': int(y + h // 2)})
    return imgs, bbs_processed

def predict(img_url, question, preprocess, model):
    img_path = os.path.join('OFA/imgs_test_public/', img_url.split('/')[-1])
    # bb_prediction = os.path.join('yolov7', 'inference', 'output', img_url.split('/')[-1]).replace('.jpg', '.txt')
    bb_prediction = os.path.join('yolov7/runs/detect/exp/labels/', img_url.split('/')[-1]).replace('.jpg', '.txt')
    imgs, bbs_processed = process_image(img_path, bb_prediction, preprocess)

    text = clip.tokenize([question]).to(device)
    probs = []
    for img in imgs:
        with torch.no_grad():
            logits_per_image, logits_per_text = model(img, text)
            probs.append(logits_per_image.softmax(dim=-1).cpu().numpy()[0][0])
    
    return bbs_processed[np.argmax(probs)]

def draw_prediction(img_url, pred_bb, gt_bb):
    img_path = os.path.join('imgs', img_url.split('/')[-1])
    img = cv2.imread(img_path)
    
    result = img.copy()
    cv2.rectangle(result, (gt_bb['x1'], gt_bb['y1']), (gt_bb['x2'], gt_bb['y2']), (0, 255, 0), 5)
    cv2.rectangle(result, (pred_bb['x1'], pred_bb['y1']), (pred_bb['x2'], pred_bb['y2']), (255, 0, 0), 5)
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
    plt.show()

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [12]:
model, preprocess = clip.load("ViT-B/32", device=device)

In [24]:
!pwd

/media/hdd/anoskov/ofa_test/yolov7


In [15]:
predictions = []
n_imgs = 0
total_iou = 0.0

progress = tqdm(test_public.iterrows(), total=len(test_public))
for _, row in progress:
    img_url = row['image']
    # print(img_url)
    # break
    question = row['question']
    try:
        bb_predicted = predict(img_url, question, preprocess, model)
    except Exception:
        print("Not predicted")
        continue
    gt_bb = {'x1': row['left'], 'y1': row['top'], 'x2': row['right'], 'y2': row['bottom']}
    total_iou += get_iou(gt_bb, bb_predicted)
    n_imgs += 1
    progress.set_description(f'IoU: {round(total_iou / n_imgs * 100, 2)}')
    
    left = bb_predicted['x1']
    top = bb_predicted['y1']
    right = bb_predicted['x2']
    bottom = bb_predicted['y2']
    predictions.append([img_url, left, top, right, bottom])
    # break
predictions = pd.DataFrame(predictions, columns=['image', 'left', 'top', 'right', 'bottom'])

IoU: 12.4:  10%|█         | 179/1705 [00:19<02:16, 11.19it/s] 

Not predicted


IoU: 12.96:  14%|█▍        | 246/1705 [00:25<03:33,  6.83it/s]

Not predicted


IoU: 12.22:  26%|██▌       | 447/1705 [00:46<02:23,  8.75it/s]

...

IoU: 11.05:  97%|█████████▋| 1655/1705 [02:52<00:03, 13.42it/s]

Not predicted


IoU: 11.05:  98%|█████████▊| 1673/1705 [02:54<00:02, 12.50it/s]

Not predicted


IoU: 11.23: 100%|██████████| 1705/1705 [02:57<00:00,  9.60it/s]


In [54]:
predictions

,image,left,top,right,bottom
0,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,128,160,216,370
1,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,20,280,70,292
2,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,389,278,405,332
3,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,359,2,431,110
4,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,605,2,639,168
...,...,...,...,...,...
1685,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,59,332,263,500
1686,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,136,240,176,284
1687,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,579,361,611,393
1688,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,227,0,259,72


In [16]:
total_iou / len(test_public) * 100

11.135882549888711